<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
# Import Pandas. Because that's how we roll
import pandas as pd
import numpy as np

In [49]:
# Get, unzip, inspect, and, if it please m'lord, load the data from lending club
# Using shell commands
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-01-14 22:14:20--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip.1’

LoanStats_2018Q4.cs     [             <=>    ]  21.72M   424KB/s    in 53s     

2020-01-14 22:15:13 (422 KB/s) - ‘LoanStats_2018Q4.csv.zip.1’ saved [22776791]



In [50]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
replace LoanStats_2018Q4.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: LoanStats_2018Q4.csv    


In [51]:
# Take a look. Just do it
!head LoanStats_2018Q4.csv

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","op

In [52]:
!tail LoanStats_2018Q4.csv

"","","5600","5600","5600"," 36 months"," 13.56%","190.21","C","C1","","n/a","RENT","15600","Not Verified","Oct-2018","Current","n","","","credit_card","Credit card refinancing","836xx","ID","15.31","0","Aug-2012","0","","97","9","1","5996","34.5%","11","w","3686.37","3686.37","2652.39","2652.39","1913.63","738.76","0.0","0.0","0.0","Dec-2019","190.21","Jan-2020","Nov-2019","0","","1","Individual","","","","0","0","5996","0","0","0","1","20","0","","0","2","3017","35","17400","1","0","0","3","750","4689","45.5","0","0","20","73","13","13","0","13","","20","","0","3","5","4","4","1","9","10","5","9","0","0","0","0","100","25","1","0","17400","5996","8600","0","","","","","","","","","","","","N","","","","","","","","","","","","","","","N","","","","","",""
"","","23000","23000","23000"," 36 months"," 15.02%","797.53","C","C3","Tax Consultant","10+ years","MORTGAGE","75000","Source Verified","Oct-2018","Charged Off","n","","","debt_consolidation","Debt consolidation","352xx","AL","20.9

In [53]:
# Two extra rows on top
# Two or three or something rows on bottom
# 2nd row, row[1], has headers
# So load using 2nd row, row[1] as header and skipfooters 2
# Without passing parameter engine=python, got warning
# # /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
#   """Entry point for launching an IPython kernel.
df = pd.read_csv('LoanStats_2018Q4.csv', header=1, skipfooter=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [54]:
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,desc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,...,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,0.0,128397.000000,128397.000000,128397.000000,128397.000000,1.283970e+05,0.0,0.0,128160.000000,128397.000000,128397.000000,56207.000000,15447.000000,128397.000000,128397.000000,1.283970e+05,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,29174.000000,128397.0,1.677800e+04,16778.000000,128397.0,128397.000000,1.283970e+05,128397.000000,128397.000000,128397.000000,128397.000000,...,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,128397.000000,125538.0,128397.0,128397.000000,128397.000000,128397.000000,126705.000000,128397.000000,128397.0,1.283970e+05,1.283970e+05,128397.000000,1.283970e+05,1.677800e+04,16778.000000,16778.000000,16778.000000,16520.000000,16778.000000,16778.000000,16778.000000,16778.000000,5152.000000,233.0,233.000000,233.0,233.000000,206.000000,233.000000,233.000000,451.000000,451.000000,451.000000
mean,NaN,NaN,15971.190332,15971.190332,15968.367147,463.248186,8.279682e+04,NaN,NaN,19.932898,0.227848,0.447035,36.879125,86.128957,11.564086,0.121841,1.689701e+04,22.677290,9530.315113,9528.904505,7596.640685,7595.021767,5805.215009,1765.306966,0.948546,25.170160,4.530629,2273.515983,0.017960,46.553369,1.0,1.335526e+05,19.225694,0.0,188.311199,1.467931e+05,0.939500,2.760181,0.689066,1.572552,...,5.414122,4.882170,7.065998,8.288262,8.199280,12.866835,5.386333,11.546750,0.0,0.0,0.059487,2.011675,94.659779,32.900216,0.121724,0.0,1.884858e+05,5.356007e+04,27439.454668,4.682151e+04,3.664349e+04,0.585946,1.587376,11.436762,55.880363,3.026821,12.425676,0.036178,0.062940,38.526592,3.0,201.703519,3.0,14.669528,607.108398,15659.828970,246.913004,7320.037317,51.851907,18.607539
std,NaN,NaN,10150.393855,10150.393855,10152.178762,285.726782,1.083035e+05,NaN,NaN,20.144436,0.733826,0.734429,21.813800,21.881192,5.981786,0.332815,2.408124e+04,12.129592,8723.959920,8724.307663,6603.469937,6603.859767,6208.300062,1483.761653,8.574509,324.692122,58.444582,5516.758598,0.146578,21.802446,0.0,9.687744e+04,8.141831,0.0,1569.377209,1.738797e+05,1.145285,2.942505,0.935777,1.565051,...,3.439678,3.205393,4.517310,7.389264,4.959021,7.874109,3.388408,5.977787,0.0,0.0,0.410667,1.880590,8.989562,34.899797,0.332542,0.0,1.965605e+05,5.599499e+04,26376.998989,4.957639e+04,3.252675e+04,0.936073,1.801935,6.690367,26.071795,3.253894,8.190168,0.347767,0.364053,23.657781,0.0,141.270219,0.0,7.821708,425.278054,9182.493729,235.871016,5027.461162,8.713731,6.838053
min,NaN,NaN,1000.000000,1000.000000,725.000000,30.480000,0.000000e+00,NaN,NaN,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.0

In [55]:
df.shape

(128397, 144)

In [56]:
df.dtypes

id                       float64
member_id                float64
loan_amnt                  int64
funded_amnt                int64
funded_amnt_inv          float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 144, dtype: object

In [57]:
# We will be working with ['term'], so let's look at that
df['term'].value_counts()


 36 months    88171
 60 months    40226
Name: term, dtype: int64

In [58]:
# Looking at first few values, seems to be either 3 year or 5 year loan term
# expressed in months
df['term'].head()

0     36 months
1     36 months
2     36 months
3     36 months
4     60 months
Name: term, dtype: object

In [59]:
# No null values
df['term'].isnull().sum()

0

In [60]:
# Getting more detail, there are leading spaces
# maybe because there might be longer terms later
df['term'][9]

' 60 months'

In [61]:
# So it's sort of a categorical string with two possible values
# we want to return integer 36 or 60
# we could do if, else or strip the whitespace
# and cast the number into an integer
# regex or...
# Strip out what we don't need
# not the most elegant, but it'll work
df['term'][1].strip().strip(' months')

'36'

In [0]:
# Still a string of course, but it works, now make a function
def term_to_int(cell_contents):
  return int(cell_contents.strip().strip(' months'))

In [63]:
# Try it once
term_to_int(df['term'][8])

36

In [0]:
# Do it all with apply()
df['term'] = df['term'].apply(term_to_int)

In [65]:
df['term'].head()

0    36
1    36
2    36
3    36
4    60
Name: term, dtype: int64

In [0]:
# Make a column named loan_status_is_great.
# It should contain the integer 1 if loan_status is "Current" or "Fully Paid."
# Else it should contain the integer 0.
# Make last_pymnt_d_month and last_pymnt_d_year columns.


# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01